# Assign overlap p values to individual proteins and perform multiple testing correction

In [1]:
import pandas as pd
import numpy as np
import statsmodels.stats.multitest

In [2]:
CHROMOSOME = "8"
ARM = "q"
TRANS_OR_CIS = "cis"

In [3]:
overlaps = pd.read_csv(f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv", sep="\t")
pvals = pd.read_csv(f"overlap_pvals_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv", sep="\t")

## Merge in the _p_ values for each protein

We will exclude proteins that were only different in one cancer type (no overlap).

In [4]:
overlaps = overlaps[overlaps["num_cancers"] > 1].\
merge(
    right=pvals,
    left_on="num_cancers",
    right_on="overlap_size",
    how="inner"
)

## Apply multiple testing correction

In [5]:
reject, adj_pvals, alpha_sidak, alpha_bonf = statsmodels.stats.multitest.multipletests(
    pvals=overlaps["pvals"], 
    alpha=0.05, 
    method="fdr_bh"
)

overlaps = overlaps.assign(
    adj_p=adj_pvals
)

In [6]:
overlaps

,protein,cancers,mean_simp_change,num_cancers,overlap_size,pvals,adj_p
0,CPNE3,brca_colon_hnscc_lscc_luad_ovarian,1,6,6,0.002510,0.015374
1,NUDCD1,brca_colon_hnscc_lscc_luad_ovarian,1,6,6,0.002510,0.015374
2,OTUD6B,brca_colon_hnscc_lscc_luad_ovarian,1,6,6,0.002510,0.015374
3,POP1,brca_colon_hnscc_lscc_luad_ovarian,1,6,6,0.002510,0.015374
4,RIDA,brca_colon_hnscc_lscc_luad_ovarian,1,6,6,0.002510,0.015374
5,RMDN1,brca_colon_hnscc_lscc_luad_ovarian,1,6,6,0.002510,0.015374
6,STK3,brca_colon_hnscc_lscc_luad_ovarian,1,6,6,0.002510,0.015374
7,YWHAZ,brca_colon_hnscc_lscc_luad_ovarian,1,6,6,0.002510,0.015374
8,FAM91A1,brca_colon_hnscc_lscc_luad,1,5,5,0.028758,0.108397
9,SQLE,brca_colon_hnscc_lscc_luad,1,5,5,0.028758,0.108397


In [7]:
overlaps.to_csv(f"sig_diff_overlaps_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv", sep="\t", index=False)